In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data= pd.read_csv('../input/machinehack-buyers-time-prediction-challenge/ParticipantData_BTPC/Train.csv')
test_data= pd.read_csv('../input/machinehack-buyers-time-prediction-challenge/ParticipantData_BTPC/Test.csv')
sub_df= pd.read_csv('../input/machinehack-buyers-time-prediction-challenge/ParticipantData_BTPC/Sample Submission.csv')

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
sub_df.head()

In [ ]:
train_data.shape, test_data.shape

In [ ]:
train= train_data.copy()
test=test_data.copy()

In [ ]:
sns.heatmap(train.corr(), annot=True, cmap='RdYlGn')

In [ ]:
train.drop(['session_id', 'client_agent'], axis=1, inplace=True)
test.drop(['session_id', 'client_agent'], axis=1, inplace=True)

In [ ]:
train.shape, test.shape

In [ ]:
train.isnull().sum()

In [ ]:
train['device_details'].value_counts()

In [ ]:
train.describe()

In [ ]:
train=train.drop_duplicates()
train.shape

In [ ]:
train["date"] = pd.to_datetime(train['date'])
test["date"] = pd.to_datetime(test['date'])

train["year"] = train['date'].dt.year
test["year"] = test['date'].dt.year

train["month"] = train['date'].dt.month
test["month"] = test['date'].dt.month

train['day']= train['date'].dt.day
test["day"] = test['date'].dt.day

train['diff_date'] = ((train['date'] - pd.datetime(1970,1,1)).dt.total_seconds())/(10**6)
test['diff_date'] = ((test['date'] - pd.datetime(1970,1,1)).dt.total_seconds())/(10**6)

In [ ]:
train.head()

In [ ]:
train.device_details.unique()

In [ ]:
train.device_details.nunique()

In [ ]:
train.dtypes

In [ ]:
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()
train["device_detail_id"]= le.fit_transform(train["device_details"])
test["device_detail_id"]= le.fit_transform(test["device_details"])

In [ ]:
train.device_detail_id.unique()

In [ ]:
train.head()

In [ ]:
plt.figure(figsize=(9,9))
sns.heatmap(train.corr(), annot=True, cmap='RdYlGn')
plt.show()

In [ ]:
train= train[pd.notnull(train["time_spent"])]

In [ ]:
train.shape, test.shape

In [ ]:
features = ['session_number','device_detail_id','purchased','added_in_cart','checked_out','year','month','day','diff_date']
X = train[features]
y = train["time_spent"]

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=1)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from statsmodels.api import OLS
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
pipelines = []
pipelines.append(('ScaledLinear', Pipeline([('Scaler', StandardScaler()),('LR',LinearRegression())])))
pipelines.append(('ScaledKNN', Pipeline([('Scaler', StandardScaler()),('KNN',KNeighborsRegressor())])))
pipelines.append(('ScaledRF', Pipeline([('Scaler', StandardScaler()),('RF',RandomForestRegressor())])))
pipelines.append(('ScaledETR', Pipeline([('Scaler', StandardScaler()),('ETR', ExtraTreesRegressor())])))
pipelines.append(('ScaledXGB', Pipeline([('Scaler', StandardScaler()),('XGB',XGBRegressor())])))
pipelines.append(('ScaledMLP', Pipeline([('Scaler', StandardScaler()),('MLP',MLPRegressor())])))
pipelines.append(('ScaledCatBoost', Pipeline([('Scaler', StandardScaler()),('CatBoost', CatBoostRegressor())])))

In [ ]:
results = []
names = []
for name, model in pipelines:
    kfold = KFold(n_splits=10, random_state=21)
    cv_results = cross_val_score(model, X, y, cv=kfold, scoring='neg_root_mean_squared_error')
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

In [ ]:
# Algorithm comparison
fig = plt.figure(figsize=(8,5))
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

In [ ]:
cboost= CatBoostRegressor(n_estimators=5000, max_depth=6).fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_log_error
pred_abs = abs(cboost.predict(X_test))
msle_score = np.sqrt(mean_squared_log_error(y_test, pred_abs))
msle_score

In [ ]:
feat_importances = pd.Series(cboost.feature_importances_, index=features)
feat_importances.nlargest(6).plot(kind='barh')
plt.show()

In [ ]:
cboost.fit(X=X[features],y=y,verbose=2)
ans = cboost.predict(test[features])

In [ ]:
ans

In [ ]:
sub_df["time_spent"]=abs(ans)
sub_df.head()

In [ ]:
sub_df.shape

In [ ]:
sub_df.to_csv('/kaggle/working/Submission_CatBoost.csv', index=False)